In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

### Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

### Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

### Data Preprocess

In [10]:
train_paths = glob('../dataset/cifar/train/*.png')[:500]
test_paths = glob('../dataset/cifar/test/*.png')[:500]

In [11]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [12]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [13]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [14]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [15]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [16]:
# AUTOTUNE = tf.data.experimental.AUTOTUNE

In [18]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
# train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(load_image_label)
# train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [19]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
# test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.map(load_image_label)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

### Callbacks

In [22]:
logdir = os.path.join('log',  datetime.now().strftime("%Y%m%d-%H%M%S"))
logdir

'log\\20201231-175606'

In [23]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir, 
    write_graph=True, 
    write_images=True,
    histogram_freq=1
)

### Training

http://localhost:6006

In [24]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard]
)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
15/15 [==============================] - 6s 219ms/step - loss: 2.3382 - accuracy: 0.1156 - val_loss: 2.3010 - val_accuracy: 0.0938
Epoch 2/10
15/15 [==============================] - 2s 122ms/step - loss: 2.2858 - accuracy: 0.1465 - val_loss: 2.2919 - val_accuracy: 0.1625
Epoch 3/10
15/15 [==============================] - 2s 120ms/step - loss: 2.2953 - accuracy: 0.1223 - val_loss: 2.2861 - val_accuracy: 0.1333
Epoch 4/10
15/15 [==============================] - 2s 122ms/step - loss: 2.2490 - accuracy: 0.2060 - val_loss: 2.1838 - val_accuracy: 0.2042
Epoch 5/10
15/15 [==============================] - 2s 122ms/step - loss: 2.1693 - accuracy: 0.1677 - val_loss: 2.2136 - val_accuracy: 0.1854
Epoch 6/10
15/15 [==============================] - 2s 119ms/step - loss: 2.2301 - accuracy: 0.2045 - val_loss: 2.1983 - val_accuracy: 0.2000
Epoch 7/10
15/15 [==============================] - 2s 131ms/step - loss: 2.1608 - accuracy: 0.2186 - val_loss: 2.1200 - val_accuracy: 0.2250
Epoch 

In [27]:
%tensorboard --logdir log

Reusing TensorBoard on port 6006 (pid 3984), started 0:00:10 ago. (Use '!kill 3984' to kill it.)

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup

### LambdaCallback

> - Confusion Matrix

In [28]:
# https://www.tensorflow.org/tensorboard/r2/image_summaries

import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [29]:
test_images, test_labels = next(iter(test_dataset))  # Confusion Matrix 그릴 때 필요한 Test Image


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_images)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [30]:
# Define the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [31]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard, cm_callback]
)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
15/15 [==============================] - 4s 162ms/step - loss: 2.0566 - accuracy: 0.2500 - val_loss: 2.0321 - val_accuracy: 0.2458


<ipython-input-28-7b8efe693ea4>:43: RuntimeWarning: invalid value encountered in true_divide
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)


Epoch 2/10
15/15 [==============================] - 2s 122ms/step - loss: 2.0750 - accuracy: 0.2372 - val_loss: 2.0998 - val_accuracy: 0.2333
Epoch 3/10
15/15 [==============================] - 2s 129ms/step - loss: 2.0926 - accuracy: 0.2417 - val_loss: 2.0620 - val_accuracy: 0.2521
Epoch 4/10
15/15 [==============================] - 2s 133ms/step - loss: 1.9653 - accuracy: 0.2697 - val_loss: 2.0072 - val_accuracy: 0.2417
Epoch 5/10
15/15 [==============================] - 2s 124ms/step - loss: 1.9078 - accuracy: 0.3013 - val_loss: 2.0143 - val_accuracy: 0.2500
Epoch 6/10
15/15 [==============================] - 2s 138ms/step - loss: 1.9171 - accuracy: 0.2778 - val_loss: 1.9345 - val_accuracy: 0.2604
Epoch 7/10
15/15 [==============================] - 2s 146ms/step - loss: 1.8705 - accuracy: 0.3063 - val_loss: 1.9187 - val_accuracy: 0.2833
Epoch 8/10
15/15 [==============================] - 2s 143ms/step - loss: 1.8190 - accuracy: 0.3141 - val_loss: 1.9147 - val_accuracy: 0.2833
Epoch 

### Expert

In [37]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [38]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [39]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)  # Loss 계산
    gradients = tape.gradient(loss, model.trainable_variables)  # 모델의 trainable_variable을 하여금 loss를 통해 기울기를 얻음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # 구한 최적화된 값을 variable에 적용

    train_loss(loss)
    train_accuracy(labels, predictions)

In [40]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [41]:
logdir = os.path.join('logs',  datetime.now().strftime("%Y%m%d-%H%M%S"))

In [42]:
file_writer = tf.summary.create_file_writer(logdir)

In [43]:
from tqdm import tqdm_notebook

In [44]:
for epoch in range(3):
    for step, (images, labels) in tqdm_notebook(enumerate(train_dataset)):
        train_step(images, labels)

    with file_writer.as_default():
        tf.summary.image('input_image', images, step=step)
        tf.summary.scalar('loss', train_loss.result(), step=step)

    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

<ipython-input-44-78e51f8a2b68>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, (images, labels) in tqdm_notebook(enumerate(train_dataset)):


KeyboardInterrupt: 